# 注:こちらはハイパーパラメータをチューニングする前の低いスコアの出るコードとなっておりますのでご注意ください！
# チューニング後のLoRA_template_unsloth2.ipynbとの比較用に置いてあります。
## コンペ用Fine-Tuningテンプレート（unsloth用）
こちらは、コンペにてFineーTuningを行いたい方に向けたテンプレートとなるFine-tuningコードです。  
Google ColabのT4でも動作可能なコードとしてunsloth版を提供します。  
環境設定の難易度が高いので、慎重に取り組んでいただければと思います。

### terminalでのconda環境構築
事前にterminalで環境構築の必要があります。  
```
# conda環境の構築
wget "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh"  

# このコマンドではいくつか質問があるので答えて下さい。おそらくインストール先のデフォルトは/root/miniforge3かと思います  
bash Miniforge3-$(uname)-$(uname -m).sh  

# 以下、インストール先が/root/miniforge3であることを前提とします  
export PATH=/root/miniforge3/bin:$PATH  
conda init  

# ここで一度、terminalを立ち上げ直す必要があります。  
# 以下のリンク先に従い環境を作ります。  
# https://docs.unsloth.ai/get-started/installation/conda-install  
conda create --name unsloth_env python=3.10 pytorch-cuda=12.1 pytorch cudatoolkit xformers -c pytorch -c nvidia -c xformers -y  
conda activate unsloth_env  
pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"  
pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes  

# jupyter notebook用のセットアップ。  
conda install -c conda-forge ipykernel  
python -m ipykernel install --user --name=unsloth_env --display-name "Python (unsloth_env)"  
```

In [ ]:
# notebookでインタラクティブな表示を可能とする（ただし、うまく動かない場合あり）
!pip install ipywidgets --upgrade

# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [ ]:
# llm-jp/llm-jp-3-13bを4bit量子化のqLoRA設定でロード。

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from unsloth import FastLanguageModel
import torch
max_seq_length = 512 # unslothではRoPEをサポートしているのでコンテキスト長は自由に設定可能
dtype = None # Noneにしておけば自動で設定
load_in_4bit = True # 今回は8Bクラスのモデルを扱うためTrue

model_id = "llm-jp/llm-jp-3-13b"
new_model_id = "llm-jp-3-13b-finetune" #Fine-Tuningしたモデルにつけたい名前
# FastLanguageModel インスタンスを作成
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    trust_remote_code=True,
)

# SFT用のモデルを用意
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 64,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
    max_seq_length = max_seq_length,
)

In [ ]:
# Hugging Faceで取得したTokenをこちらに貼る。
HF_TOKEN = "Write権限のあるHF Token"

In [ ]:
"""
dataset: 学習に用いるデータセット

  ベースコードでは以下のリンクからデータをダウンロードして使います。zipを展開(!unzip)してデータのパスを指定してください。
  (https://liat-aip.sakura.ne.jp/wp/llmのための日本語インストラクションデータ作成/llmのための日本語インストラクションデータ-公開/)
  関根聡, 安藤まや, 後藤美知子, 鈴木久美, 河原大輔, 井之上直也, 乾健太郎.
  ichikara-instruction: LLMのための日本語インストラクションデータの構築. 言語処理学会第30回年次大会(2024)

omnicampusの開発環境では取得したデータを左側にドラッグアンドドロップしてお使いください。
"""
from datasets import load_dataset

dataset = load_dataset("json", data_files="./ichikara-instruction-003-001-1.json")
dataset

In [ ]:
# 学習時のプロンプトフォーマットの定義
prompt = """### 指示
{}
### 回答
{}"""



"""
formatting_prompts_func: 各データをプロンプトに合わせた形式に合わせる
"""
EOS_TOKEN = tokenizer.eos_token # トークナイザーのEOSトークン（文末トークン）
def formatting_prompts_func(examples):
    input = examples["text"] # 入力データ
    output = examples["output"] # 出力データ
    text = prompt.format(input, output) + EOS_TOKEN # プロンプトの作成
    return { "formatted_text" : text, } # 新しいフィールド "formatted_text" を返す
pass

# # 各データにフォーマットを適用
dataset = dataset.map(
    formatting_prompts_func,
    num_proc= 4, # 並列処理数を指定
)

dataset

In [ ]:
# データを確認
print(dataset["train"]["formatted_text"][3])

In [12]:
# データをtrainデータとtestデータに分割 (test_sizeの比率に)
dataset = dataset["train"].train_test_split(test_size=0.1)
dataset

In [ ]:
"""
training_arguments: 学習の設定

  - output_dir:
      -トレーニング後のモデルを保存するディレクトリ

  - per_device_train_batch_size:
      - デバイスごとのトレーニングバッチサイズ

  - per_device_eval_batch_size:
      - デバイスごとの評価バッチサイズ

  - gradient_accumulation_steps:
      - 勾配を更新する前にステップを積み重ねる回数

  - optim:
      - オプティマイザの設定

  - num_train_epochs:
      - エポック数

  - eval_strategy:
      - 評価の戦略 ("no"/"steps"/"epoch")

  - eval_steps:
      - eval_strategyが"steps"のとき、評価を行うstep間隔

  - logging_strategy:
      - ログ記録の戦略

  - logging_steps:
      - ログを出力するステップ間隔

  - warmup_steps:
      - 学習率のウォームアップステップ数

  - save_steps:
      - モデルを保存するステップ間隔

  - save_total_limit:
      - 保存しておくcheckpointの数

  - max_steps:
      - トレーニングの最大ステップ数

  - learning_rate:
      - 学習率

  - fp16:
      - 16bit浮動小数点の使用設定（第8回演習を参考にすると良いです）

  - bf16:
      - BFloat16の使用設定

  - group_by_length:
      -  入力シーケンスの長さによりバッチをグループ化 (トレーニングの効率化)

  - report_to:
      - ログの送信先 ("wandb"/"tensorboard"など)
"""
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    max_seq_length = max_seq_length,
    dataset_text_field="formatted_text",
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        per_device_eval_batch_size = 1,
        gradient_accumulation_steps = 4,
        optim = "paged_adamw_32bit",
        num_train_epochs = 1,
        eval_steps=0.2,
        logging_steps = 10,
        warmup_steps = 10,
        save_steps=100,
        save_total_limit=2,
        max_steps=-1,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        group_by_length=True,
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#@title 現在のメモリ使用量を表示
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
#@title 学習実行
trainer_stats = trainer.train()

In [ ]:
# モデルとトークナイザーをHugging Faceにアップロード。  
# 一旦privateでアップロードしてください。  
# 最終成果物が決まったらpublicにするようお願いします。  
model.push_to_hub_merged(new_model_id, token=HF_TOKEN, private=True) # Online saving  
tokenizer.push_to_hub(new_model_id, token=HF_TOKEN) # Online saving  